<a href="https://colab.research.google.com/github/cheyoung211/2023-Auttum-Maths-for-ML/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대회 데이터가 일부 변경되었습니다.
# 아래 경로의 Competition Site에 오시면 비슷한 데이터로 실습하실 수 있습니다.<br>
https://www.kaggle.com/competitions/original-instant-gratification

In [ ]:
#모듈 불러오기
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold

from sklearn.decomposition import KernelPCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgbm

In [ ]:
class hist_model(object):

    def __init__(self, bins=50):
        self.bins = bins

    def fit(self, X):

        bin_hight, bin_edge = [], []

        for var in X.T:
            # get bins hight and interval
            bh, bedge = np.histogram(var, bins=self.bins)
            bin_hight.append(bh)
            bin_edge.append(bedge)

        self.bin_hight = np.array(bin_hight)
        self.bin_edge = np.array(bin_edge)

    def predict(self, X):

        scores = []
        for obs in X:
            obs_score = []
            for i, var in enumerate(obs):
                # find wich bin obs is in
                bin_num = (var > self.bin_edge[i]).argmin()-1
                obs_score.append(self.bin_hight[i, bin_num]) # find bin hitght

            scores.append(np.mean(obs_score))

        return np.array(scores)

In [ ]:
def run_model(clf_list, train, test, random_state, gmm_init_params='kmeans'):

    MODEL_COUNT = len(clf_list)

    oof_train = np.zeros((len(train), MODEL_COUNT)) #새로 생성되는 train data를 저장
    oof_test = np.zeros((len(test), MODEL_COUNT)) # 새로 생성되는 test data를 저장
    train_columns = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']] #column 이름 변경

    for magic in tqdm_notebook(range(512)):
        x_train = train[train['wheezy-copper-turtle-magic'] == magic]
        x_test = test[test['wheezy-copper-turtle-magic'] == magic]
        print("Magic: ", magic, x_train.shape, x_test.shape)

        train_idx_origin = x_train.index
        test_idx_origin = x_test.index

        train_std = x_train[train_columns].std()
        cols = list(train_std.index.values[np.where(train_std >2)])

        x_train = x_train.reset_index(drop=True)
        y_train = x_train.target

        x_train = x_train[cols].values
        x_test = x_test[cols].values

        all_data = np.vstack([x_train, x_test])
        # print("all_data: ", all_data.shape)
        # Kernel PCA
        all_data = KernelPCA(n_components=len(cols), kernel='cosine', random_state=random_state).fit_transform(all_data)

        # GMM
        gmm = GMM(n_components=5, random_state=random_state, max_iter=1000, init_params=gmm_init_params).fit(all_data)
        gmm_pred = gmm.predict_proba(all_data)
        gmm_score = gmm.score_samples(all_data).reshape(-1, 1)
        gmm_label = gmm.predict(all_data)

        # hist feature
        hist = hist_model()
        hist.fit(all_data)
        hist_pred = hist.predict(all_data).reshape(-1, 1)

        all_data = np.hstack([all_data, gmm_pred, gmm_pred, gmm_pred, gmm_pred, gmm_pred])

        # Add Some Features
        all_data = np.hstack([all_data, hist_pred, gmm_score, gmm_score, gmm_score])

        # STANDARD SCALER
        all_data = StandardScaler().fit_transform(all_data) #scaling

        # new train/test - 새로운 train, test 데이터
        x_train = all_data[:x_train.shape[0]]
        x_test = all_data[x_train.shape[0]:]
        # print("data size: ", x_train.shape, x_test.shape)
        fold = StratifiedKFold(n_splits=5, random_state=random_state) #fold 나누기
        for trn_idx, val_idx in fold.split(x_train, gmm_label[:x_train.shape[0]]):
            #모델 인덱스와 모델을 가져오는 loop
            for model_index, clf in enumerate(clf_list):
                clf.fit(x_train[trn_idx], y_train[trn_idx]) #각 모델을 fit
                oof_train[train_idx_origin[val_idx], model_index] = clf.predict_proba(x_train[val_idx])[:,1] #새로운 train 데이터 생성

                # 2023/03/02 데이터의 형식이 변경되어, x_test 예측 시 오류 발생하는 것 수정
                if x_test.shape[0] == 0:
                    continue

                #print(oof_test[test_idx_origin, model_index].shape)
                #print(x_test.shape)
                #print(clf.predict_proba(x_test)[:,1])
                oof_test[test_idx_origin, model_index] += clf.predict_proba(x_test)[:,1] / fold.n_splits

    #train된 모델을 인덱스와 함께 가져옴
    for i, clf in enumerate(clf_list):
        print(clf) #모델
        print(roc_auc_score(train['target'], oof_train[:, i])) #학습 결과
        print()

    oof_train_df = pd.DataFrame(oof_train)
    oof_test_df = pd.DataFrame(oof_test)

    return oof_train_df, oof_test_df

In [ ]:
os.listdir('../input/instant-gratification/')

['sample_submission.csv', 'train.csv', 'test.csv']

In [ ]:
train = pd.read_csv('../input/instant-gratification/train.csv')
test = pd.read_csv('../input/instant-gratification/test.csv')

In [ ]:
#parameter 설정
svnu_params = {'probability':True, 'kernel':'poly','degree':4,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
svnu2_params = {'probability':True, 'kernel':'poly','degree':2,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
qda_params = {'reg_param':0.111}
svc_params = {'probability':True,'kernel':'poly','degree':4,'gamma':'auto', 'random_state':4}
neighbor_params = {'n_neighbors':16}
lr_params = {'solver':'liblinear','penalty':'l1','C':0.05,'random_state':42}

In [ ]:
#생성된 모델에 parameter 적용
nusvc_model = svm.NuSVC(**svnu_params)
nusvc2_model = svm.NuSVC(**svnu2_params)
qda_model = QuadraticDiscriminantAnalysis(**qda_params)
svc_model = svm.SVC(**svc_params)
knn_model = neighbors.KNeighborsClassifier(**neighbor_params)
lr_model = linear_model.LogisticRegression(**lr_params)

#emsemble에 사용할 모델 list에 넣기
#순서대로 training한 뒤 생성된 train, test 데이터 저장
model_list = [nusvc_model, nusvc2_model, qda_model, svc_model, knn_model, lr_model]
oof_train_kmeans_seed1, oof_test_kmeans_seed1 = run_model(model_list, train, test, 1)
oof_train_kmeans_seed2, oof_test_kmeans_seed2 = run_model(model_list, train, test, 2)
oof_train_random_seed1, oof_test_random_seed1 = run_model(model_list, train, test, 1, 'random')
oof_train_random_seed2, oof_test_random_seed2 = run_model(model_list, train, test, 2, 'random')

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

In [ ]:
#2번째 stage
#각 train, test 데이터 합쳐서 새로운 dataset 만들기
train_second = (oof_train_kmeans_seed1 + oof_train_kmeans_seed2 + oof_train_random_seed1 + oof_train_random_seed2)/4
test_second = (oof_test_kmeans_seed1 + oof_test_kmeans_seed2 + oof_test_random_seed1 + oof_test_random_seed2)/4
print('Ensemble', roc_auc_score(train['target'], train_second.mean(1)))

Ensemble 0.9658510356167946


In [ ]:
lgbm_meta_param = {
        #'bagging_freq': 5,
        #'bagging_fraction': 0.8,
        'min_child_weight':6.790,
        "subsample_for_bin":50000,
        'bagging_seed': 0,
        'boost_from_average':'true',
        'boost': 'gbdt',
        'feature_fraction': 0.450,
        'bagging_fraction': 0.343,
        'learning_rate': 0.025,
        'max_depth': 10,
        'metric':'auc',
        'min_data_in_leaf': 78,
        'min_sum_hessian_in_leaf': 8,
        'num_leaves': 18,
        'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary',
        'verbosity': 1,
        'lambda_l1': 7.961,
        'lambda_l2': 7.781
        #'reg_lambda': 0.3,
    }

mlp16_params = {'activation':'relu','solver':'lbfgs','tol':1e-06, 'hidden_layer_sizes':(16, ), 'random_state':42}

In [ ]:
SEED_NUMBER = 4
NFOLD = 5

#새로운 train, test data 준비용
y_train = train['target']
oof_lgbm_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_lgbm_meta_test = np.zeros((len(test), SEED_NUMBER))
oof_mlp_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_mlp_meta_test = np.zeros((len(test), SEED_NUMBER))

for seed in range(SEED_NUMBER):
    print("SEED Ensemble:", seed)
    mlp16_params['random_state'] = seed
    lgbm_meta_param['seed'] = seed
    folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=seed)
    #train 진행
    for fold_index, (trn_index, val_index) in enumerate(folds.split(train_second, y_train), 1):
        print(f"{fold_index} FOLD Start")
        trn_x, trn_y = train_second.iloc[trn_index], y_train.iloc[trn_index]
        val_x, val_y = train_second.iloc[val_index], y_train.iloc[val_index]

        mlp_meta_model = neural_network.MLPClassifier(**mlp16_params)
        mlp_meta_model.fit(trn_x, trn_y)

        oof_mlp_meta_train[val_index, seed] = mlp_meta_model.predict_proba(val_x)[:,1]
        oof_mlp_meta_test[:, seed] += mlp_meta_model.predict_proba(test_second)[:,1]/NFOLD
        print("MLP META SCORE: ", roc_auc_score(val_y, oof_mlp_meta_train[val_index, seed]))

        # lgbm meta model
        dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
        dcross = lgbm.Dataset(val_x, label=val_y, silent=True)

        lgbm_meta_model = lgbm.train(lgbm_meta_param, train_set=dtrain, valid_sets=[dtrain, dcross],
                                     verbose_eval=False, early_stopping_rounds=100)

        #model predict 결과를 train, test data로 삼기
        oof_lgbm_meta_train[val_index, seed] = lgbm_meta_model.predict(val_x)
        oof_lgbm_meta_test[:, seed] += lgbm_meta_model.predict(test_second)/NFOLD
        print("LGBM META SCORE: ", roc_auc_score(val_y, oof_lgbm_meta_train[val_index, seed]))

SEED Ensemble: 0
1 FOLD Start
MLP META SCORE:  0.9679149157241265
LGBM META SCORE:  0.967647530635912
2 FOLD Start
MLP META SCORE:  0.9680669908384355
LGBM META SCORE:  0.9674288751838224
3 FOLD Start
MLP META SCORE:  0.968603479583272
LGBM META SCORE:  0.9683328443489624
4 FOLD Start
MLP META SCORE:  0.9686502145269664
LGBM META SCORE:  0.9682168798889179
5 FOLD Start
MLP META SCORE:  0.9686257557500182
LGBM META SCORE:  0.968488858766663
SEED Ensemble: 1
1 FOLD Start
MLP META SCORE:  0.9694567291526788
LGBM META SCORE:  0.9689072412786429
2 FOLD Start
MLP META SCORE:  0.9684907321825804
LGBM META SCORE:  0.967816212999968
3 FOLD Start
MLP META SCORE:  0.9676346435556265
LGBM META SCORE:  0.9671898995884282
4 FOLD Start
MLP META SCORE:  0.9694087333144282
LGBM META SCORE:  0.9689640788253138
5 FOLD Start
MLP META SCORE:  0.9686281947259526
LGBM META SCORE:  0.9679262404257606
SEED Ensemble: 2
1 FOLD Start
MLP META SCORE:  0.967660503571859
LGBM META SCORE:  0.9673343320936969
2 FOLD S

In [ ]:
#학습결과에서 새로운 data 생성
oof_lgbm_meta_train_df = pd.DataFrame(oof_lgbm_meta_train).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_lgbm_meta_test_df = pd.DataFrame(oof_lgbm_meta_test).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_mlp_meta_train_df = pd.DataFrame(oof_mlp_meta_train).mean(axis=1).to_frame().rename(columns={0:'mlp'})
oof_mlp_meta_test_df = pd.DataFrame(oof_mlp_meta_test).mean(axis=1).to_frame().rename(columns={0:'mlp'})

In [ ]:
#3번째 stage
#앞서 생성된 data를 concat
oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)

print('Ensemble', roc_auc_score(train['target'], oof_train_third.mean(1))) #최종 emsemble 결과

Ensemble 0.9671407107739252


In [ ]:
submission = pd.read_csv('../input/instant-gratification/sample_submission.csv')
submission["target"] = oof_test_third.mean(1)
submission.to_csv("submission.csv", index=False) #csv로 저장